In [16]:
from en_test import dataset, labels
from en_xlm_roberta_model import *

In [19]:
en_test = pd.read_csv('/home/mlepekhin/data/en_train')
labels = np.unique(sorted(en_test.target.values))

label_to_id = {label: label_id for label_id, label in enumerate(labels)}
id_to_label = {label_id: label for label_id, label in enumerate(labels)}

In [18]:
print(id_to_label)

{0: 'A1', 1: 'A11', 2: 'A12', 3: 'A14', 4: 'A16', 5: 'A17', 6: 'A4', 7: 'A7', 8: 'A8', 9: 'A9'}


In [3]:
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder

from textattack.attack_recipes import TextFoolerJin2019
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding
from textattack.shared.attack import Attack

from textattack.shared import WordEmbedding

In [4]:
import numpy as np
import pickle

embedding_matrix = np.load('en_fasttext_30000/embeddings_matrix.npy')
nn_matrix = np.load('en_fasttext_30000/nn_matrix.npy')
index2word = pickle.load(open('en_fasttext_30000/index2word.pcl', 'rb'))
word2index = pickle.load(open('en_fasttext_30000/word2index.pcl', 'rb'))

In [5]:
import collections

smart_word2index = collections.defaultdict(int, word2index)
smart_index2word = collections.defaultdict(str, index2word)

In [6]:
en_fasttext_embedding = WordEmbedding(embedding_matrix, word2index, index2word, nn_matrix)

In [7]:
def build_textfooler(model):
    #
    # Swap words with their 50 closest embedding nearest-neighbors.
    # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
    #
    transformation = WordSwapEmbedding(max_candidates=50, embedding=en_fasttext_embedding)
    #
    # Don't modify the same word twice or the stopwords defined
    # in the TextFooler public implementation.
    #
    # fmt: off
    # fmt: on
    constraints = [RepeatModification()]
    #
    # During entailment, we should only edit the hypothesis - keep the premise
    # the same.
    #
    input_column_modification = InputColumnModification(
        ["premise", "hypothesis"], {"premise"}
    )
    constraints.append(input_column_modification)
    # Minimum word embedding cosine similarity of 0.5.
    # (The paper claims 0.7, but analysis of the released code and some empirical
    # results show that it's 0.5.)
    #
    
    constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
    
    #
    # Only replace words with the same part of speech (or nouns with verbs)
    #
    
    #constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
    
    #
    # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
    #
    # In the TextFooler code, they forget to divide the angle between the two
    # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
    # new threshold is 1 - (0.5) / pi = 0.840845057
    #
    use_constraint = UniversalSentenceEncoder(
        threshold=0.84,
        metric="angular",
        compare_against_original=False,
        window_size=15,
        skip_text_shorter_than_window=True,
    )
    constraints.append(use_constraint)
    #
    # Goal is untargeted classification
    #
    goal_function = UntargetedClassification(model)
    #
    # Greedily swap words with "Word Importance Ranking".
    #
    search_method = GreedyWordSwapWIR(wir_method="delete")

    return Attack(goal_function, constraints, transformation, search_method)

In [8]:
#recipe = TextFoolerJin2019(
#    goal_function=UntargetedClassification(model), 
#    search_method=GreedyWordSwapWIR(wir_method="delete"),
#    transformation=WordSwapEmbedding(max_candidates=50),
#)
#recipe.constraints = constraints

In [9]:
import pandas as pd

result_df = pd.DataFrame({'attack_result': [], 'cross_val_batch': []})

In [10]:
dataset_names = [f'en_train_crossval_{fold_id}' for fold_id in range(5)]
model_names = [f'xlm_roberta_model_en_crossval_{fold_id}' for fold_id in range(5)]

In [11]:
from tqdm import tqdm

for batch_id, (dataset_name, model_name) in enumerate(zip(dataset_names, model_names)):
    cur_dataset = __import__(dataset_name).dataset
    cur_model = __import__(model_name).model
    attacker = build_textfooler(cur_model)
    for idx, result in tqdm(enumerate(attacker.attack_dataset(cur_dataset))):
        result_df = result_df.append({
            'attack_result': result.__str__(color_method='html'),
            'cross_val_batch': batch_id
        }, ignore_index=True)
        #print(('x' * 20), f'Result {idx+1}', ('x' * 20))
        #print(result.__str__(color_method='html'))
        #print(len(str(result).split('\n\n')))
    result_df.to_csv('en_xlm_roberta_cross_val_result_50.csv')
    del cur_model


11 LABELS


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

11 LABELS


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

11 LABELS


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

11 LABELS


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

11 LABELS


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [12]:
print(len(result_df))

1264


In [13]:
def remove_color_tags(text):
    text = re.sub(r'<font color = [a-zA-Z]+>', ' ', text)
    text = re.sub(r'</font>', ' ', text)
    text = text.replace('  ', ' ')
    return text

In [20]:
all_targets = []

for dataset_name in dataset_names:
    dataset_labels = [id_to_label[pair[1]] for pair in __import__(dataset_name).dataset]
    all_targets = all_targets + dataset_labels
print(all_targets)

['A12', 'A4', 'A1', 'A1', 'A12', 'A9', 'A12', 'A1', 'A8', 'A14', 'A11', 'A22', 'A4', 'A7', 'A14', 'A14', 'A22', 'A8', 'A14', 'A12', 'A22', 'A1', 'A9', 'A1', 'A12', 'A7', 'A16', 'A12', 'A7', 'A12', 'A4', 'A1', 'A1', 'A16', 'A7', 'A8', 'A11', 'A1', 'A8', 'A12', 'A12', 'A9', 'A1', 'A1', 'A1', 'A16', 'A7', 'A9', 'A12', 'A7', 'A8', 'A7', 'A4', 'A12', 'A11', 'A1', 'A8', 'A4', 'A12', 'A17', 'A7', 'A7', 'A7', 'A1', 'A22', 'A16', 'A14', 'A4', 'A1', 'A1', 'A17', 'A16', 'A9', 'A1', 'A7', 'A8', 'A8', 'A22', 'A1', 'A16', 'A7', 'A8', 'A1', 'A9', 'A22', 'A14', 'A14', 'A12', 'A12', 'A16', 'A12', 'A4', 'A8', 'A16', 'A1', 'A9', 'A14', 'A1', 'A4', 'A1', 'A4', 'A9', 'A9', 'A1', 'A22', 'A7', 'A7', 'A16', 'A16', 'A16', 'A7', 'A1', 'A7', 'A11', 'A12', 'A1', 'A16', 'A7', 'A7', 'A8', 'A1', 'A8', 'A16', 'A7', 'A14', 'A14', 'A12', 'A11', 'A4', 'A7', 'A11', 'A9', 'A1', 'A12', 'A12', 'A1', 'A9', 'A8', 'A16', 'A12', 'A16', 'A7', 'A7', 'A4', 'A1', 'A8', 'A16', 'A1', 'A7', 'A11', 'A7', 'A12', 'A8', 'A11', 'A1', 'A1',

In [21]:
import re

ru_attacked_df = pd.DataFrame()
attack_count = collections.defaultdict(int)
word_count = collections.defaultdict(list)
good_count = 0

for raw_text, real_target in zip(result_df.attack_result, all_targets):
    if len(raw_text.split('\n\n')) == 3:
        header, old, new = raw_text.split('\n\n')
        _, label_id1, label_id2 = re.split(r'<font color = [a-zA-Z]+>', header)
        label1 = id_to_label[int(label_id1.split()[0])]
        label2 = id_to_label[int(label_id2.split()[0])]
        changed_words = len(tuple(re.finditer(r'<font color = [a-zA-Z]+>', old)))
        word_count[(label1, label2)].append(changed_words)
        attack_count[(label1, label2)] += 1
        
        
        ru_attacked_df = ru_attacked_df.append(
            {'old_text': remove_color_tags(old), 
             'text': remove_color_tags(new),
             'changed_words_num': changed_words,
             'old_model_target': label1,
             'new_model_target': label2,
             'target': real_target
            },
            ignore_index=True
        )
        good_count += 1
print(good_count)

clean_stat_df = pd.DataFrame()
for (label1, label2) in attack_count.keys():
    cur_attack_count = attack_count[(label1, label2)]
    cur_word_count = word_count[(label1, label2)]
    clean_stat_df = clean_stat_df.append(
        {'model_label_old': label1, 
         'model_label_new': label2, 
         'mean_words': sum(cur_word_count) / cur_attack_count, 
         'median': np.median(cur_word_count)
        }, 
        ignore_index=True
    )
clean_stat_df.head()

453


,mean_words,median,model_label_new,model_label_old
0,46.222222,39.0,A22,A1
1,20.761905,6.0,A12,A1
2,32.250000,28.5,A1,A9
3,29.575000,21.0,A8,A1
4,18.750000,11.0,A1,A8


In [22]:
clean_stat_df.to_csv('/home/mlepekhin/data/en_textfooler_stat_50.csv')

In [23]:
ru_attacked_df.head()

,changed_words_num,new_model_target,old_model_target,old_text,target,text
0,31.0,A22,A1,"Come gather round me comrades , and hear my l...",A1,"Come assemble round me friend , both listenin..."
1,1.0,A12,A1,"Grades 7 - 12 , Upper School Each year Beth Da...",A1,"Grades 7 - 12 , Upper School Each year Beth Da..."
2,53.0,A1,A9,WHEREAS by the Wimbledon and Putney Commons A...,A9,BUT by the Wimbledon and Biggleswade Commons ...
3,41.0,A8,A1,"Can I just say , having been named by Timothy ...",A1,"Can me just tell , having been appointed by Ti..."
4,5.0,A1,A8,"For Goldman Sachs , a public-relations disaste...",A8,"For Savitzky Sachs , a public-relations disast..."


In [24]:
ru_attacked_df.to_csv('/home/mlepekhin/data/en_attacked_50.csv')

In [25]:
remove_color_tags('<font color = gray>Нижнекамскнефтехим')

' Нижнекамскнефтехим'